In [57]:
import geopandas as gpd
import os
import pandas as pd
import pathlib
import warnings
warnings.filterwarnings("ignore")

#from osgeo import gdal
from shapely.geometry import Point

Voir : https://www.data.gouv.fr/fr/pages/donnees-geographiques/#:~:text=URL%3A%20https%3A%2F%2Fwww.data.gouv.fr%2Ffr%2Fpages%2Fdonnees

## Premières features

In [58]:
#dir_geo = pathlib.Path() / 'data' / 'geolocalisation'
dir_geo = pathlib.Path('../data/geolocalisation')
DEPARTEMENT = '78'

In [59]:
df_geom = gpd.read_file(dir_geo / 'geometries' / 'geometries.gpkg')

In [60]:
df_geom['surface'] = df_geom['geometry'].area

In [61]:
df_geom['longitude'] = df_geom['geometry'].centroid.x
df_geom['latitude'] = df_geom['geometry'].centroid.y

In [62]:
df_geom

,type,nom,code_geom,geometry,surface,longitude,latitude
0,DEP,DOUBS,0,"POLYGON ((5.69876 47.26464, 5.69877 47.26481, ...",0.62361,6.362722,47.165964


## Features au niveau des IRIS

### Lecture des IRIS du département

In [63]:
# Fichier de contours IRIS : https://geoservices.ign.fr/contoursiris
geoiris = gpd.read_file(dir_geo / 'iris' / 'CONTOURS-IRIS.shp')

In [64]:
# Coordinate Reference System : Lambert 93
geoiris.crs = "EPSG:2154"

In [65]:
geoiris = geoiris.loc[geoiris.INSEE_COM.str.startswith(DEPARTEMENT)]

In [66]:
geoiris

,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS,geometry
9,78474,Orvilliers,0000,784740000,Orvilliers,Z,"POLYGON ((600495.600 6864316.200, 600586.900 6..."
45,78401,Meulan-en-Yvelines,0104,784010104,Bords de Seine,H,"POLYGON ((621312.300 6878774.500, 621117.200 6..."
81,78361,Mantes-la-Jolie,0106,783610106,Millet,H,"POLYGON ((603651.700 6879070.200, 603654.600 6..."
83,78231,Favrieux,0000,782310000,Favrieux,Z,"POLYGON ((599969.700 6873207.000, 600254.800 6..."
123,78362,Mantes-la-Ville,0108,783620108,Les Orgemonts,H,"MULTIPOLYGON (((605072.100 6875514.100, 605142..."
...,...,...,...,...,...,...,...
48024,78356,Magny-les-Hameaux,0103,783560103,Cressely et Buisson 2,H,"POLYGON ((633413.300 6847546.100, 633472.800 6..."
48052,78528,Rolleboise,0000,785280000,Rolleboise,Z,"POLYGON ((596754.500 6881123.600, 596756.200 6..."
48081,78440,Les Mureaux,0109,784400109,Vigne Blanche 1,H,"POLYGON ((620517.300 6875936.300, 620516.600 6..."
48151,78172,Conflans-Sainte-Honorine,0109,781720109,Centre Ville,H,"POLYGON ((634246.700 6877611.600, 634266.300 6..."


In [67]:
geoiris.shape

(683, 7)

In [68]:
for fic in ["base-ic-activite-residents-2019.xlsx", 
            "base-ic-couples-familles-menages-2019.xlsx", 
            "base-ic-diplomes-formation-2019.xlsx",
            "base-ic-evol-struct-pop-2019.xlsx",
            "base-ic-logement-2019.xlsx"]:
    df = pd.read_excel(dir_geo / 'insee' / fic, header = 4, nrows=10)
    df.dropna(axis=1, inplace=True)
    dico = dict(zip(list(df.head(1).values[0]), df.columns))
    for k in dico:
        if k.startswith(('P19', 'C19')):
            print(f'"{k}", # {dico[k]}')
            #if k=='C19_MEN':
            #    print(fic)

"P19_POP1564", # Pop 15-64 ans en 2019 (princ)
"P19_POP1524", # Pop 15-24 ans en 2019 (princ)
"P19_POP2554", # Pop 25-54 ans en 2019 (princ)
"P19_POP5564", # Pop 55-64 ans en 2019 (princ)
"P19_H1564", # Pop 15-64 ans Hommes en 2019 (princ)
"P19_H1524", # Pop 15-24 ans Hommes en 2019 (princ)
"P19_H2554", # Pop 25-54 ans Hommes en 2019 (princ)
"P19_H5564", # Pop 55-64 ans Hommes en 2019 (princ)
"P19_F1564", # Pop 15-64 ans Femmes en 2019 (princ)
"P19_F1524", # Pop 15-24 ans Femmes en 2019 (princ)
"P19_F2554", # Pop 25-54 ans Femmes en 2019 (princ)
"P19_F5564", # Pop 55-64 ans Femmes en 2019 (princ)
"P19_ACT1564", # Actifs 15-64 ans en 2019 (princ)
"P19_ACT1524", # Actifs 15-24 ans en 2019 (princ)
"P19_ACT2554", # Actifs 25-54 ans en 2019 (princ)
"P19_ACT5564", # Actifs 55-64 ans en 2019 (princ)
"P19_HACT1564", # Actifs 15-64 ans Hommes en 2019 (princ)
"P19_HACT1524", # Actifs 15-24 ans Hommes en 2019 (princ)
"P19_HACT2554", # Actifs 25-54 ans Hommes en 2019 (princ)
"P19_HACT5564", # Acti

### Base de données infracommunale : logement

On s'occupe, pour un IRIS donné, de :
- son nombre de logements,    le nombre de résidences principales,
- le nombre de résidences secondaires et logements occasionnels,
- le nombre de logements vacants. À partir du fichier récupéré ici : 

https://www.insee.fr/fr/statistiques?debut=0&idprec=6543302&categorie=1&geo=ICQ-1
Plus précisément, ici :
https://www.insee.fr/fr/statistiques/6543302

In [69]:
df = pd.read_excel(dir_geo / 'insee' / "base-ic-logement-2019.xlsx", header = 5)

In [70]:
df.rename(columns={'IRIS':'CODE_IRIS'}, inplace=True)
df.dropna(axis=1, inplace=True)

In [71]:
geoiris = geoiris.merge(df, how = 'left') 

In [72]:
geoiris.loc[geoiris.NOM_COM == 'Chatou'].T

,23,91,100,129,187,277,281,331,485,551,582,657,682
INSEE_COM,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146
NOM_COM,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou
IRIS,0110,0105,0109,0106,0104,0111,0112,0107,0108,0103,0113,0101,0102
CODE_IRIS,781460110,781460105,781460109,781460106,781460104,781460111,781460112,781460107,781460108,781460103,781460113,781460101,781460102
NOM_IRIS,La Piece d'Eau,Maupassant,Hôtel de Ville,Europe,Ancien Cimetière,Brimont,Berteaux,Les Marolles,Gambetta,Vlaminck,Labelonye,Tete de Girafe,Les Landes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
P19_RP_VOIT1P,797.808573,908.36218,647.963447,1276.844308,953.64016,681.639918,763.887579,641.883722,791.345423,757.001018,637.972694,1002.515271,786.271922
P19_RP_VOIT1,516.978563,682.235713,444.123329,855.85023,590.634308,491.552258,602.605217,428.996632,493.953637,509.126709,393.534173,545.0315,565.81089
P19_RP_VOIT2P,280.830009,226.126467,203.840118,420.994078,363.005852,190.087659,161.282362,212.88709,297.391786,247.874308,244.438522,457.48377,220.461032
C19_RP_HSTU1P,898.468074,975.6759,726.489181,1360.374003,1059.493106,764.796627,944.436378,908.716929,885.876585,829.041547,753.343807,1084.812587,951.702007


In [73]:
geoiris

,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS,geometry,REG,DEP,UU2020,...,P19_ANEM_RP_PROP,P19_ANEM_RP_LOC,P19_ANEM_RP_LOCHLMV,P19_ANEM_RP_GRAT,P19_RP_GARL,P19_RP_VOIT1P,P19_RP_VOIT1,P19_RP_VOIT2P,C19_RP_HSTU1P,C19_RP_HSTU1P_SUROCC
0,78474,Orvilliers,0000,784740000,Orvilliers,Z,"POLYGON ((600495.600 6864316.200, 600586.900 6...",11,78,78000,...,3848.911717,277.402214,0.000000,77.516961,295.938042,323.959607,115.501354,208.458253,329.929431,15.011407
1,78401,Meulan-en-Yvelines,0104,784010104,Bords de Seine,H,"POLYGON ((621312.300 6878774.500, 621117.200 6...",11,78,00851,...,6262.098036,5557.725279,2158.155773,465.709307,607.102842,1088.106932,853.474029,234.632903,1232.913554,150.312441
2,78361,Mantes-la-Jolie,0106,783610106,Millet,H,"POLYGON ((603651.700 6879070.200, 603654.600 6...",11,78,00851,...,0.000000,4359.568062,4250.707807,12.341200,29.597111,161.656229,137.481645,24.174584,284.998691,73.696433
3,78231,Favrieux,0000,782310000,Favrieux,Z,"POLYGON ((599969.700 6873207.000, 600254.800 6...",11,78,78000,...,1158.000000,29.000000,0.000000,138.000000,61.000000,65.000000,26.000000,39.000000,65.000000,0.000000
4,78362,Mantes-la-Ville,0108,783620108,Les Orgemonts,H,"MULTIPOLYGON (((605072.100 6875514.100, 605142...",11,78,00851,...,13585.204027,2550.293485,1306.618986,200.578581,702.640594,844.024360,521.979711,322.044649,955.588689,56.628258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,78356,Magny-les-Hameaux,0103,783560103,Cressely et Buisson 2,H,"POLYGON ((633413.300 6847546.100, 633472.800 6...",11,78,00851,...,8893.206985,3071.541583,2845.728646,99.063716,630.749375,647.810160,237.149887,410.660273,670.606667,25.054043
679,78528,Rolleboise,0000,785280000,Rolleboise,Z,"POLYGON ((596754.500 6881123.600, 596756.200 6...",11,78,78302,...,2361.962771,385.879022,220.546306,44.646194,113.535362,155.648510,84.425355,71.223155,154.745042,5.038579
680,78440,Les Mureaux,0109,784400109,Vigne Blanche 1,H,"POLYGON ((620517.300 6875936.300, 620516.600 6...",11,78,00851,...,56.888213,3286.693726,3150.345633,27.015856,92.121172,188.718818,159.586399,29.132419,297.767883,69.715376
681,78172,Conflans-Sainte-Honorine,0109,781720109,Centre Ville,H,"POLYGON ((634246.700 6877611.600, 634266.300 6...",11,78,00851,...,9548.337660,6331.719157,3070.910174,102.462260,562.043974,920.403935,647.434899,272.969036,1047.311969,111.975823


### Base de données infracommunale : population

La base infracommunale « Population » fournit des données sur les caractéristiques de la population selon le sexe, l'âge, la catégorie socio-professionnelle et la catégorie de nationalité.
https://www.insee.fr/fr/statistiques/6543200

In [74]:
df = pd.read_excel(dir_geo / 'insee' / "base-ic-evol-struct-pop-2019.xlsx", header = 5)

In [75]:
df.shape

(49282, 84)

In [76]:
df.rename(columns={'IRIS':'CODE_IRIS'}, inplace=True)

In [77]:
geoiris = geoiris.merge(df, how = 'left') 
geoiris.dropna(axis=1, inplace=True)

In [78]:
geoiris.loc[geoiris.NOM_COM == 'Chatou'].T

,23,91,100,129,187,277,281,331,485,551,582,657,682
INSEE_COM,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146
NOM_COM,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou
IRIS,0110,0105,0109,0106,0104,0111,0112,0107,0108,0103,0113,0101,0102
CODE_IRIS,781460110,781460105,781460109,781460106,781460104,781460111,781460112,781460107,781460108,781460103,781460113,781460101,781460102
NOM_IRIS,La Piece d'Eau,Maupassant,Hôtel de Ville,Europe,Ancien Cimetière,Brimont,Berteaux,Les Marolles,Gambetta,Vlaminck,Labelonye,Tete de Girafe,Les Landes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
C19_F15P_CS8,151.821922,143.963383,124.765129,229.760251,191.171368,150.82644,169.168167,208.830346,138.227145,115.192093,214.350472,258.008995,201.301579
P19_POP_FR,2130.819439,1996.787495,1748.824476,3066.836788,2602.831456,1728.453303,1947.734374,1927.409413,1944.340625,1854.300132,1792.713952,2670.636923,2239.063027
P19_POP_ETR,182.06302,301.379908,62.516906,310.479014,199.967002,71.758854,145.466658,354.126386,155.725296,247.760484,88.529233,160.869128,221.606707
P19_POP_IMM,272.731778,418.227685,130.536683,476.197835,301.00452,144.527047,215.669354,501.588799,282.558131,336.4276,139.912776,307.738538,450.381967


### Activité des résidents

In [79]:
df = pd.read_excel(dir_geo / 'insee' / "base-ic-activite-residents-2019.xlsx", header = 5)

In [80]:
df.shape

(49282, 127)

In [81]:
df.rename(columns={'IRIS':'CODE_IRIS'}, inplace=True)
geoiris = geoiris.merge(df, how = 'left') 
geoiris.dropna(axis=1, inplace=True)

In [84]:
geoiris.loc[geoiris.NOM_COM == 'Chatou'].T

,23,91,100,129,187,277,281,331,485,551,582,657,682
INSEE_COM,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146
NOM_COM,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou
IRIS,0110,0105,0109,0106,0104,0111,0112,0107,0108,0103,0113,0101,0102
CODE_IRIS,781460110,781460105,781460109,781460106,781460104,781460111,781460112,781460107,781460108,781460103,781460113,781460101,781460102
NOM_IRIS,La Piece d'Eau,Maupassant,Hôtel de Ville,Europe,Ancien Cimetière,Brimont,Berteaux,Les Marolles,Gambetta,Vlaminck,Labelonye,Tete de Girafe,Les Landes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
C19_ACTOCC15P_MAR,85.624637,65.856532,37.137494,92.598707,69.018181,57.363277,53.971391,79.275933,55.454403,61.228981,35.625893,19.416856,90.376408
C19_ACTOCC15P_VELO,42.032055,36.264497,16.284375,47.179544,49.581226,21.115889,12.681327,47.355638,27.497436,27.009014,12.168635,50.044807,23.551659
C19_ACTOCC15P_2ROUESMOT,69.183004,52.684541,32.573399,90.217744,62.262986,21.562516,34.727514,23.080286,38.845354,80.6881,48.391986,48.893369,30.27512
C19_ACTOCC15P_VOIT,341.868532,534.062898,236.490415,814.395618,566.501328,314.494626,287.026607,398.386036,420.454911,545.470445,264.713984,684.017464,485.073711


### Diplômes - Formation

In [85]:
df = pd.read_excel(dir_geo / 'insee' / "base-ic-diplomes-formation-2019.xlsx", header = 5)

In [86]:
df.rename(columns={'IRIS':'CODE_IRIS'}, inplace=True)
geoiris = geoiris.merge(df, how = 'left') 
geoiris.dropna(axis=1, inplace=True)

In [87]:
geoiris.loc[geoiris.NOM_COM == 'Chatou'].T

,23,91,100,129,187,277,281,331,485,551,582,657,682
INSEE_COM,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146
NOM_COM,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou
IRIS,0110,0105,0109,0106,0104,0111,0112,0107,0108,0103,0113,0101,0102
CODE_IRIS,781460110,781460105,781460109,781460106,781460104,781460111,781460112,781460107,781460108,781460103,781460113,781460101,781460102
NOM_IRIS,La Piece d'Eau,Maupassant,Hôtel de Ville,Europe,Ancien Cimetière,Brimont,Berteaux,Les Marolles,Gambetta,Vlaminck,Labelonye,Tete de Girafe,Les Landes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
P19_FNSCOL15P_CAPBEP,22.712893,120.486411,43.312444,136.408026,142.541641,76.612557,53.870267,166.544973,105.446925,116.73685,19.771089,164.892438,124.146106
P19_FNSCOL15P_BAC,111.635144,134.637855,75.156777,216.317232,197.081759,110.255403,86.676654,173.096537,127.531468,144.156549,75.657451,143.227806,166.98248
P19_FNSCOL15P_SUP2,107.831429,128.514679,85.667328,218.428509,154.854018,82.684159,115.856517,72.307358,121.117204,157.061962,80.917502,157.735496,149.580305
P19_FNSCOL15P_SUP34,162.621772,131.450316,164.094351,211.892986,173.482653,141.625777,145.342127,103.689667,152.224242,143.772538,150.828581,188.245168,117.111755


### Couples, familles, ménages

In [88]:
df = pd.read_excel(dir_geo / 'insee' / "base-ic-couples-familles-menages-2019.xlsx", header = 5)

In [89]:
df.rename(columns={'IRIS':'CODE_IRIS'}, inplace=True)
geoiris = geoiris.merge(df, how = 'left') 
geoiris.dropna(axis=1, inplace=True)

In [91]:
geoiris.loc[geoiris.NOM_COM == 'Chatou'].T

,23,91,100,129,187,277,281,331,485,551,582,657,682
INSEE_COM,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146
NOM_COM,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou
IRIS,0110,0105,0109,0106,0104,0111,0112,0107,0108,0103,0113,0101,0102
CODE_IRIS,781460110,781460105,781460109,781460106,781460104,781460111,781460112,781460107,781460108,781460103,781460113,781460101,781460102
NOM_IRIS,La Piece d'Eau,Maupassant,Hôtel de Ville,Europe,Ancien Cimetière,Brimont,Berteaux,Les Marolles,Gambetta,Vlaminck,Labelonye,Tete de Girafe,Les Landes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
P19_FNSCOL15P_CAPBEP,22.712893,120.486411,43.312444,136.408026,142.541641,76.612557,53.870267,166.544973,105.446925,116.73685,19.771089,164.892438,124.146106
P19_FNSCOL15P_BAC,111.635144,134.637855,75.156777,216.317232,197.081759,110.255403,86.676654,173.096537,127.531468,144.156549,75.657451,143.227806,166.98248
P19_FNSCOL15P_SUP2,107.831429,128.514679,85.667328,218.428509,154.854018,82.684159,115.856517,72.307358,121.117204,157.061962,80.917502,157.735496,149.580305
P19_FNSCOL15P_SUP34,162.621772,131.450316,164.094351,211.892986,173.482653,141.625777,145.342127,103.689667,152.224242,143.772538,150.828581,188.245168,117.111755


### Dénombrement des équipements (commerce, services, santé...)

https://www.insee.fr/fr/statistiques/3568629?sommaire=3568656

In [92]:
df = pd.read_csv(dir_geo / 'insee' / "bpe21_ensemble.csv", sep=';')#, header = 5)

In [93]:
df.rename(columns={'DCIRIS':'CODE_IRIS'}, inplace=True)

In [94]:
df = df.loc[df.DEP==eval(DEPARTEMENT)]

In [95]:
for classe in ['DOM', 'SDOM', 'TYPEQU']:
    dg = df[['CODE_IRIS', classe, 'NB_EQUIP']]
    # Grouper par CODE_IRIS et DOM, et sommer les NB_EQUIP
    grouped = dg.groupby(['CODE_IRIS', classe]).sum()
    # Pivoter pour obtenir DOM en colonnes
    pivot = grouped['NB_EQUIP'].unstack(fill_value=0)
    # Réinitialiser l'index pour avoir CODE_IRIS comme colonne
    pivot_reset = pivot.reset_index()
    # Renommer les colonnes si nécessaire
    pivot_reset.columns.name = None     
    # Afficher ou utiliser le nouveau DataFrame
    pivot_reset.columns = ['CODE_IRIS']+[f'equipement_{k}' for k in list(pivot_reset.columns) if 'CODE' not in k]
    geoiris = geoiris.merge(pivot_reset, how = 'left') 
    geoiris.dropna(axis=0, inplace=True)

In [96]:
[k for k in geoiris if "D106" in k]

['equipement_D106']

In [97]:
geoiris.loc[geoiris.NOM_COM == 'Chatou'].T

,23,91,100,129,187,276,280,330,484,550,581,656,681
INSEE_COM,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146,78146
NOM_COM,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou,Chatou
IRIS,0110,0105,0109,0106,0104,0111,0112,0107,0108,0103,0113,0101,0102
CODE_IRIS,781460110,781460105,781460109,781460106,781460104,781460111,781460112,781460107,781460108,781460103,781460113,781460101,781460102
NOM_IRIS,La Piece d'Eau,Maupassant,Hôtel de Ville,Europe,Ancien Cimetière,Brimont,Berteaux,Les Marolles,Gambetta,Vlaminck,Labelonye,Tete de Girafe,Les Landes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
equipement_F315,0,0,0,0,0,0,1,0,0,0,0,0,0
equipement_G101,1,1,0,0,2,0,1,0,0,1,2,0,0
equipement_G102,0,0,0,0,0,0,0,0,0,0,0,1,0
equipement_G103,0,0,0,0,0,0,0,0,0,0,0,0,0


## IGN

### BDforet

https://geoservices.ign.fr/bdforet

In [98]:
df = gpd.read_file(dir_geo / 'BDforet' / DEPARTEMENT / 'FORMATION_VEGETALE.shp')

DriverError: ../data/geolocalisation/BDforet/78/FORMATION_VEGETALE.shp: No such file or directory

In [37]:
for var in df.TFV.unique():
    print(var)
    df_geom[f'surface_{var}'] = 0.0
    dg = df.loc[df.TFV==var]
    for index, geom_row in df_geom.iterrows():
        surface_totale = 0.0
        for _, df_row in dg.iterrows():
            # Calculer l'intersection
            intersection = geom_row.geometry.intersection(df_row.geometry)
            # Vérifier si l'intersection est non vide et calculer la surface
            if not intersection.is_empty:
                surface_totale += intersection.area
        # Assigner la surface totale d'intersection à la ligne correspondante dans df_geom
        df_geom.at[index, f'surface_{var}'] = surface_totale

Lande
Forêt fermée sans couvert arboré
Forêt fermée de feuillus purs en îlots
Forêt fermée à mélange de feuillus
Forêt fermée à mélange de feuillus prépondérants et conifères
Forêt fermée de hêtre pur
Forêt fermée de châtaignier pur
Forêt fermée de robinier pur
Forêt fermée d’un autre feuillu pur
Forêt fermée à mélange de conifères prépondérants et feuillus
Forêt fermée de chênes décidus purs
Forêt fermée de conifères purs en îlots
Forêt fermée à mélange de conifères
Forêt fermée de pin maritime pur
Forêt fermée de pin sylvestre pur
Forêt fermée de mélèze pur
Forêt fermée de douglas pur
Forêt fermée à mélange de pins purs
Forêt fermée à mélange d’autres conifères
Forêt fermée de pin laricio ou pin noir pur
Forêt fermée de sapin ou épicéa
Forêt ouverte de feuillus purs
Forêt ouverte sans couvert arboré
Forêt ouverte de conifères purs
Forêt ouverte à mélange de feuillus et conifères
Peupleraie


In [38]:
for var in df.TFV_G11.unique():
    if f'surface_{var}' not in df_geom:
        print(var)
        df_geom[f'surface_{var}'] = 0.0
        dg = df.loc[df.TFV_G11==var]
        for index, geom_row in df_geom.iterrows():
            surface_totale = 0.0
            for _, df_row in dg.iterrows():
                # Calculer l'intersection
                intersection = geom_row.geometry.intersection(df_row.geometry)
                # Vérifier si l'intersection est non vide et calculer la surface
                if not intersection.is_empty:
                    surface_totale += intersection.area
            # Assigner la surface totale d'intersection à la ligne correspondante dans df_geom
            df_geom.at[index, f'surface_{var}'] = surface_totale

Forêt fermée feuillus
Forêt fermée mixte
Forêt fermée conifères
Forêt ouverte feuillus
Forêt ouverte conifères
Forêt ouverte mixte


In [39]:
for var in df.ESSENCE.unique():
    if f'surface_{var}' not in df_geom and var not in ['NC', 'NR']:
        print(var)
        df_geom[f'surface_{var}'] = 0.0
        dg = df.loc[df.ESSENCE==var]
        for index, geom_row in df_geom.iterrows():
            surface_totale = 0.0
            for _, df_row in dg.iterrows():
                # Calculer l'intersection
                intersection = geom_row.geometry.intersection(df_row.geometry)
                # Vérifier si l'intersection est non vide et calculer la surface
                if not intersection.is_empty:
                    surface_totale += intersection.area
            # Assigner la surface totale d'intersection à la ligne correspondante dans df_geom
            df_geom.at[index, f'surface_{var}'] = surface_totale

Feuillus
Mixte
Hêtre
Châtaignier
Robinier
Chênes décidus
Conifères
Pin maritime
Pin sylvestre
Mélèze
Douglas
Pins mélangés
Pin laricio, pin noir
Sapin, épicéa
Peuplier


## Les accidents

https://www.data.gouv.fr/fr/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2022/#/resources

In [40]:
df = pd.DataFrame()
for annee in range(2017, 2023):
    if annee <= 2018:
        df0 = pd.read_csv(dir_geo / 'Accidents' / f'caracteristiques-{annee}.csv', sep=',', encoding='ISO-8859-1')
        dh0 = pd.read_csv(dir_geo / 'Accidents' / f'usagers-{annee}.csv', sep=',', encoding='ISO-8859-1')
    else:
        df0 = pd.read_csv(dir_geo / 'Accidents' / f'caracteristiques-{annee}.csv', sep=';')
        dh0 = pd.read_csv(dir_geo / 'Accidents' / f'usagers-{annee}.csv', sep=';')
    if df0['long'].dtype == 'O':
        df0['long'] = df0['long'].str.replace(',', '.').astype(float)
        df0['lat'] = df0['lat'].str.replace(',', '.').astype(float)
    else:
        df0['long'] /= 10**5
        df0['lat'] /= 10**5
    if 'Num_Acc' not in df0:
        df0.rename({'Accident_Id': 'Num_Acc'}, axis=1, inplace=True)
    df_merged = pd.merge(df0, dh0[['Num_Acc', 'grav']], on='Num_Acc', how='left')
    df_merged = df_merged.assign(year=annee)
    df_merged['day'] = df_merged['jour']
    df_merged['month'] = df_merged['mois']
    df_merged['date'] = pd.to_datetime(df_merged[['year', 'month', 'day']])
    df_merged['jour_semaine'] = df_merged['date'].dt.dayofweek  # ou .dt.day_name() pour le nom
    df = pd.concat([df, df_merged])
    
df['geometry'] = [Point(xy) for xy in zip(df['long'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.set_crs(epsg=4326, inplace=True)
gdf = gdf.to_crs(epsg=2154)
join = gpd.sjoin(df_geom, gdf, how='left', op='contains')
count_accidents = join.groupby(join.index).size()
df_geom['accidents'] = count_accidents
df_geom['accidents'].fillna(0, inplace=True)
for gravite in range(1,4):
    gdf1 = gdf.loc[gdf.grav == gravite]
    join = gpd.sjoin(df_geom, gdf, how='left', op='contains')
    count_accidents = join.groupby(join.index).size()
    df_geom[f'accidents_grav{gravite}'] = count_accidents
    df_geom[f'accidents_grav{gravite}'].fillna(0, inplace=True)
MOIS = ['jan', 'fev', 'mars', 'avr', 'mai', 'juin', 'juil', 'aout', 'sep', 'oct', 'nov', 'dec']
for mois in range(1, 13):
    gdf0 = gdf.loc[gdf.mois==mois]
    join = gpd.sjoin(df_geom, gdf0, how='left', op='contains')
    count_accidents = join.groupby(join.index).size()
    df_geom[f'accidents_{MOIS[mois-1]}'] = count_accidents
    df_geom[f'accidents_{MOIS[mois-1]}'].fillna(0, inplace=True)
    for gravite in range(1,4):
        gdf1 = gdf0.loc[gdf0.grav == gravite]
        join = gpd.sjoin(df_geom, gdf1, how='left', op='contains')
        count_accidents = join.groupby(join.index).size()
        df_geom[f'accidents_{MOIS[mois-1]}_grav{gravite}'] = count_accidents
        df_geom[f'accidents_{MOIS[mois-1]}_grav{gravite}'].fillna(0, inplace=True)
JOUR = ['lun', 'mar', 'mer', 'jeu', 'ven', 'sam', 'dim']
for jour_semaine in range(7):
    gdf0 = gdf.loc[gdf.jour_semaine==jour_semaine]
    join = gpd.sjoin(df_geom, gdf0, how='left', op='contains')
    count_accidents = join.groupby(join.index).size()
    df_geom[f'accidents_{JOUR[jour_semaine]}'] = count_accidents
    df_geom[f'accidents_{JOUR[jour_semaine]}'].fillna(0, inplace=True)
    for gravite in range(1,4):
        gdf1 = gdf0.loc[gdf0.grav == gravite]
        join = gpd.sjoin(df_geom, gdf1, how='left', op='contains')
        count_accidents = join.groupby(join.index).size()
        df_geom[f'accidents_{JOUR[jour_semaine]}_grav{gravite}'] = count_accidents
        df_geom[f'accidents_{JOUR[jour_semaine]}_grav{gravite}'].fillna(0, inplace=True)

## Ajout de l'altitude moyenne

## Intégration à nos géométries

In [41]:
variables = [
"P19_POP1564", # Pop 15-64 ans en 2019 (princ)
"P19_POP1524", # Pop 15-24 ans en 2019 (princ)
"P19_POP2554", # Pop 25-54 ans en 2019 (princ)
"P19_POP5564", # Pop 55-64 ans en 2019 (princ)
"P19_H1564", # Pop 15-64 ans Hommes en 2019 (princ)
"P19_H1524", # Pop 15-24 ans Hommes en 2019 (princ)
"P19_H2554", # Pop 25-54 ans Hommes en 2019 (princ)
"P19_H5564", # Pop 55-64 ans Hommes en 2019 (princ)
"P19_F1564", # Pop 15-64 ans Femmes en 2019 (princ)
"P19_F1524", # Pop 15-24 ans Femmes en 2019 (princ)
"P19_F2554", # Pop 25-54 ans Femmes en 2019 (princ)
"P19_F5564", # Pop 55-64 ans Femmes en 2019 (princ)
"P19_ACT1564", # Actifs 15-64 ans en 2019 (princ)
"P19_ACT1524", # Actifs 15-24 ans en 2019 (princ)
"P19_ACT2554", # Actifs 25-54 ans en 2019 (princ)
"P19_ACT5564", # Actifs 55-64 ans en 2019 (princ)
"P19_HACT1564", # Actifs 15-64 ans Hommes en 2019 (princ)
"P19_HACT1524", # Actifs 15-24 ans Hommes en 2019 (princ)
"P19_HACT2554", # Actifs 25-54 ans Hommes en 2019 (princ)
"P19_HACT5564", # Actifs 55-64 ans Hommes en 2019 (princ)
"P19_FACT1564", # Actifs 15-64 ans Femmes en 2019 (princ)
"P19_FACT1524", # Actifs 15-24 ans Femmes en 2019 (princ)
"P19_FACT2554", # Actifs 25-54 ans Femmes en 2019 (princ)
"P19_FACT5564", # Actifs 55-64 ans Femmes en 2019 (princ)
"P19_ACTOCC1564", # Actifs occupés 15-64 ans en 2019 (princ)
"P19_ACTOCC1524", # Actifs occupés 15-24 ans en 2019 (princ)
"P19_ACTOCC2554", # Actifs occupés 25-54 ans en 2019 (princ)
"P19_ACTOCC5564", # Actifs occupés 55-64 ans en 2019 (princ)
"P19_HACTOCC1564", # Actifs occupés 15-64 ans Hommes en 2019 (princ)
"P19_HACTOCC1524", # Actifs occupés 15-24 ans Hommes en 2019 (princ)
"P19_HACTOCC2554", # Actifs occupés 25-54 ans Hommes en 2019 (princ)
"P19_HACTOCC5564", # Actifs occupés 55-64 ans Hommes en 2019 (princ)
"P19_FACTOCC1564", # Actifs occupés 15-64 ans Femmes en 2019 (princ)
"P19_FACTOCC1524", # Actifs occupés 15-24 ans Femmes en 2019 (princ)
"P19_FACTOCC2554", # Actifs occupés 25-54 ans Femmes en 2019 (princ)
"P19_CHOM_CAPBEP", # Chômeurs CAP-BEP ou équiv. en 2019 (princ)
"P19_FACTOCC5564", # Actifs occupés 55-64 ans Femmes en 2019 (princ)
"P19_CHOM_BAC", # Chômeurs Bac, brevet pro. ou équiv.  en 2019 (princ)
"P19_CHOM1564", # Chômeurs 15-64 ans en 2019 (princ)
"P19_CHOM_SUP2", # Chômeurs Enseignement sup de niveau bac + 2  en 2019 (princ)
"P19_CHOM1524", # Chômeurs 15-24 ans en 2019 (princ)
"P19_CHOM_SUP34", # Chômeurs Enseignement sup de niveau bac + 3 ou 4  en 2019 (princ)
"P19_CHOM2554", # Chômeurs 25-54 ans en 2019 (princ)
"P19_CHOM_SUP5", # Chômeurs Enseignement sup de niveau bac + 5 ou plus  en 2019 (princ)
"P19_CHOM5564", # Chômeurs 55-64 ans en 2019 (princ)
"P19_ACT_DIPLMIN", # Actifs Sans diplôme ou CEP en 2019 (princ)
"P19_CHOM_DIPLMIN", # Chômeurs Sans diplôme ou CEP en 2019 (princ)
"P19_ACT_BEPC", # Actifs BEPC, brevet des collèges, DNB en 2019 (princ)
"P19_CHOM_BEPC", # Chômeurs BEPC, brevet des collèges, DNB en 2019 (princ)
"P19_ACT_CAPBEP", # Actifs CAP-BEP ou équiv. en 2019 (princ)
"P19_ACT_BAC", # Actifs Bac, brevet pro. ou équiv.  en 2019 (princ)
"P19_ACT_SUP2", # Actifs Enseignement sup de niveau bac + 2  en 2019 (princ)
"P19_ACT_SUP34", # Actifs Enseignement sup de niveau bac + 3 ou 4  en 2019 (princ)
"P19_ACT_SUP5", # Actifs Enseignement sup de niveau bac + 5 ou plus  en 2019 (princ)
"P19_INACT1564", # Inactifs 15-64 ans en 2019 (princ)
"P19_HINACT1564", # Inactifs 15-64 ans Hommes en 2019 (princ)
"P19_FINACT1564", # Inactifs 15-64 ans Femmes en 2019 (princ)
"P19_ETUD1564", # Elèv. Etud. Stag. non rémunérés 15-64 ans en 2019 (princ)
"P19_HETUD1564", # Elèv. Etud. Stag. non rémunérés 15-64 ans Hommes en 2019 (princ)
"P19_FETUD1564", # Elèv. Etud. Stag. non rémunérés 15-64 ans Femmes en 2019 (princ)
"P19_RETR1564", # Retraités Préretraités 15-64 ans en 2019 (princ)
"P19_HRETR1564", # Retraités Préretraités 15-64 ans Hommes en 2019 (princ)
"P19_FRETR1564", # Retraités Préretraités 15-64 ans Femmes en 2019 (princ)
"P19_AINACT1564", # Autres inactifs 15-64 ans en 2019 (princ)
"P19_HAINACT1564", # Autres inactifs 15-64 ans Hommes en 2019 (princ)
"P19_FAINACT1564", # Autres inactifs 15-64 ans Femmes en 2019 (princ)
"C19_ACT1564", # Actifs 15-64 ans en 2019 (compl)
"C19_ACT1564_CS1", # Actifs 15-64 ans Agriculteurs exploitants en 2019 (compl)
"C19_ACT1564_CS2", # Actifs 15-64 ans Artisans, Comm., Chefs entr. en 2019 (compl)
"C19_ACT1564_CS3", # Actifs 15-64 ans Cadres, Prof. intel. sup. en 2019 (compl)
"C19_ACT1564_CS4", # Actifs 15-64 ans Prof. intermédiaires en 2019 (compl)
"C19_ACT1564_CS5", # Actifs 15-64 ans Employés en 2019 (compl)
"C19_ACT1564_CS6", # Actifs 15-64 ans Ouvriers en 2019 (compl)
"C19_ACTOCC1564", # Actifs occupés 15-64 ans en 2019 (compl)
"C19_ACTOCC1564_CS1", # Actifs occ 15-64 ans Agriculteurs exploitants en 2019 (compl)
"C19_ACTOCC1564_CS2", # Actifs occ 15-64 ans Artisans, Comm., Chefs entr. en 2019 (compl)
"C19_ACTOCC1564_CS3", # Actifs occ 15-64 ans Cadres Prof. intel. sup. en 2019 (compl)
"C19_ACTOCC1564_CS4", # Actifs occ 15-64 ans Prof. intermédiaires en 2019 (compl)
"C19_ACTOCC1564_CS5", # Actifs occupés 15-64 ans Employés en 2019 (compl)
"C19_ACTOCC1564_CS6", # Actifs occupés 15-64 ans Ouvriers en 2019 (compl)
"P19_ACTOCC15P", # Actifs occupés 15 ans ou plus en 2019 (princ)
"P19_HACTOCC15P", # Actifs occupés 15 ans ou plus Hommes en 2019 (princ)
"P19_FACTOCC15P", # Actifs occupés 15 ans ou plus Femmes en 2019 (princ)
"P19_SAL15P", # Salariés 15 ans ou plus en 2019 (princ)
"P19_HSAL15P", # Salariés 15 ans ou plus Hommes en 2019 (princ)
"P19_FSAL15P", # Salariés 15 ans ou plus Femmes en 2019 (princ)
"P19_NSAL15P", # Non-salariés 15 ans ou plus en 2019 (princ)
"P19_HNSAL15P", # Non-salariés 15 ans ou plus Hommes en 2019 (princ)
"P19_FNSAL15P", # Non-salariés 15 ans ou plus Femmes en 2019 (princ)
"P19_ACTOCC15P_TP", # Actifs occ 15 ans ou plus TP en 2019 (princ)
"P19_SAL15P_TP", # Salariés 15 ans ou plus TP en 2019 (princ)
"P19_HSAL15P_TP", # Salariés 15 ans ou plus TP Hommes en 2019 (princ)
"P19_FSAL15P_TP", # Salariés 15 ans ou plus TP Femmes en 2019 (princ)
"P19_NSAL15P_TP", # Non-salariés 15 ans ou plus TP en 2019 (princ)
"P19_SAL15P_CDI", # Salariés 15 ans ou plus Fonct publ, CDI en 2019 (princ)
"P19_SAL15P_CDD", # Salariés 15 ans ou plus CDD en 2019 (princ)
"P19_SAL15P_INTERIM", # Salariés 15 ans ou plus Intérim en 2019 (princ)
"P19_SAL15P_EMPAID", # Salariés 15 ans ou plus Emplois aidés en 2019 (princ)
"P19_SAL15P_APPR", # Salariés 15 ans ou plus Apprentissage - Stage en 2019 (princ)
"P19_NSAL15P_INDEP", # Non-salariés 15 ans ou plus Indépendants en 2019 (princ)
"P19_NSAL15P_EMPLOY", # Non-salariés 15 ans ou plus Employeurs en 2019 (princ)
"P19_NSAL15P_AIDFAM", # Non-salariés 15 ans ou plus Aides familiaux en 2019 (princ)
"P19_ACTOCC15P_ILT1", # Actif occ 15 ans ou plus travaille commune résidence en 2019 (princ)
"P19_ACTOCC15P_ILT2P", # Actif occ 15 ans ou plus travaille autre commune que commune résidence en 2019 (princ)
"P19_ACTOCC15P_ILT2", # Actif occ 15 ans ou plus travaille autre commune même dépt résidence en 2019 (princ)
"P19_ACTOCC15P_ILT3", # Actif occ 15 ans ou plus travaille autre dépt même région résidence en 2019 (princ)
"P19_ACTOCC15P_ILT4", # Actif occ 15 ans ou plus travaille autre région en métropole en 2019 (princ)
"P19_ACTOCC15P_ILT5", # Actif occ 15 ans ou plus travaille autre région hors métropole en 2019 (princ)
"C19_ACTOCC15P", # Actif occ 15 ans ou plus en 2019 (compl)
"C19_ACTOCC15P_PAS", # Actifs occ 15 ans ou plus pas de transport pour travail en 2019 (compl)
"C19_ACTOCC15P_MAR", # Actifs occ 15 ans ou plus marche à pied pour travail en 2019 (compl)
"C19_ACTOCC15P_VELO", # Actifs occ 15 ans ou plus vélo en 2019 (compl)
"C19_ACTOCC15P_2ROUESMOT", # Actifs occ 15 ans ou plus deux-roues motorisé en 2019 (compl)
"C19_ACTOCC15P_VOIT", # Actifs occ 15 ans ou plus voiture en 2019 (compl)
"C19_ACTOCC15P_TCOM", # Actifs occ 15 ans ou plus transport en commun en 2019 (compl)
"C19_MEN", # Ménages en 2019 (compl)
"C19_MENPSEUL", # Ménages 1 personne en 2019 (compl)
"C19_MENHSEUL", # Ménages Hommes seuls en 2019 (compl)
"C19_MENFSEUL", # Ménages Femmes seules en 2019 (compl)
"C19_MENSFAM", # Ménages Autres sans famille en 2019 (compl)
"C19_MENFAM", # Ménages avec famille(s) en 2019 (compl)
"C19_MENCOUPSENF", # Mén fam princ Couple sans enfant en 2019 (compl)
"C19_MENCOUPAENF", # Mén fam princ Couple avec enfant(s) en 2019 (compl)
"C19_MENFAMMONO", # Mén fam princ Famille mono en 2019 (compl)
"C19_PMEN", # Pop Ménages en 2019 (compl)
"C19_PMEN_MENPSEUL", # Pop mén Personnes seules en 2019 (compl)
"C19_PMEN_MENHSEUL", # Pop mén Hommes seuls en 2019 (compl)
"C19_PMEN_MENFSEUL", # Pop mén Femmes seules en 2019 (compl)
"C19_PMEN_MENSFAM", # Pop mén Autres sans famille en 2019 (compl)
"C19_PMEN_MENFAM", # Pop mén avec famille(s) en 2019 (compl)
"C19_PMEN_MENCOUPSENF", # Pop mén fam princ Couple sans enfant en 2019 (compl)
"C19_PMEN_MENCOUPAENF", # Pop mén fam princ Couple avec enfant(s) en 2019 (compl)
"C19_PMEN_MENFAMMONO", # Pop mén fam princ Famille mono en 2019 (compl)
"P19_POP15P", # Pop 15 ans ou plus en 2019 (princ)
"P19_POP1524", # Pop 15-24 ans en 2019 (princ)
"P19_POP2554", # Pop 25-54 ans en 2019 (princ)
"P19_POP5579", # Pop 55-79 ans en 2019 (princ)
"P19_POP80P", # Pop 80 ans ou plus en 2019 (princ)
"P19_POPMEN15P", # Pop mén 15 ans ou plus en 2019 (princ)
"P19_POPMEN1524", # Pop mén 15-24 ans en 2019 (princ)
"P19_POPMEN2554", # Pop mén 25-54 ans en 2019 (princ)
"P19_POPMEN5579", # Pop mén 55-79 ans en 2019 (princ)
"P19_POPMEN80P", # Pop mén 80 ans ou plus en 2019 (princ)
"P19_POP15P_PSEUL", # Pop 15 ans ou plus ans vivant seule en 2019 (princ)
"P19_POP1524_PSEUL", # Pop 15-24 ans vivant seule en 2019 (princ)
"P19_POP2554_PSEUL", # Pop 25-54 ans vivant seule en 2019 (princ)
"P19_POP5579_PSEUL", # Pop 55-79 ans vivant seule en 2019 (princ)
"P19_POP80P_PSEUL", # Pop 80 ans ou plus vivant seule en 2019 (princ)
"P19_POP15P_MARIEE", # Pop 15 ans ou plus mariée en 2019 (princ)
"P19_POP15P_PACSEE", # Pop 15 ans ou plus pacsée en 2019 (princ)
"P19_POP15P_CONCUB_UNION_LIBRE", # Pop 15 ans ou plus en concubinage ou union libre en 2019 (princ)
"P19_POP15P_VEUFS", # Pop 15 ans ou plus veuves ou veufs en 2019 (princ)
"P19_POP15P_DIVORCEE", # Pop 15 ans ou plus divorcée en 2019 (princ)
"P19_POP15P_CELIBATAIRE", # Pop 15 ans ou plus célibataire en 2019 (princ)
"C19_MEN_CS1", # Ménages Pers Réf Agri. expl. en 2019 (compl)
"C19_MEN_CS2", # Ménages Pers Réf Art. Comm. Chef entr. en 2019 (compl)
"C19_MEN_CS3", # Ménages Pers Réf Cadre Prof int sup en 2019 (compl)
"C19_MEN_CS4", # Ménages Pers Réf Prof intermédiaire en 2019 (compl)
"C19_MEN_CS5", # Ménages Pers Réf Employé en 2019 (compl)
"C19_MEN_CS6", # Ménages Pers Réf Ouvrier en 2019 (compl)
"C19_MEN_CS7", # Ménages Pers Réf Retraité en 2019 (compl)
"C19_MEN_CS8", # Ménages Pers Réf Autre en 2019 (compl)
"C19_PMEN_CS1", # Pop mén Pers Réf Agri. expl. en 2019 (compl)
"C19_PMEN_CS2", # Pop mén Pers Réf Art Com Chef entr. en 2019 (compl)
"C19_PMEN_CS3", # Pop mén Pers Réf Cadre Prof int sup en 2019 (compl)
"C19_PMEN_CS4", # Pop mén Pers Réf Prof intermédiaire en 2019 (compl)
"C19_PMEN_CS5", # Pop mén Pers Réf Employé en 2019 (compl)
"C19_PMEN_CS6", # Pop mén Pers Réf Ouvrier en 2019 (compl)
"C19_PMEN_CS7", # Pop mén Pers Réf Retraité en 2019 (compl)
"C19_PMEN_CS8", # Pop mén Pers Réf Autre en 2019 (compl)
"C19_FAM", # Familles en 2019 (compl)
"C19_COUPAENF", # Fam Couple avec enfant(s) en 2019 (compl)
"C19_FAMMONO", # Fam Monoparentales en 2019 (compl)
"C19_COUPSENF", # Fam Couple sans enfant en 2019 (compl)
"C19_NE24F0", # Fam 0 enfant moins 25 ans en 2019 (compl)
"C19_NE24F1", # Fam 1 enfant moins 25 ans en 2019 (compl)
"C19_NE24F2", # Fam 2 enfants moins 25 ans en 2019 (compl)
"C19_NE24F3", # Fam 3 enfants moins 25 ans en 2019 (compl)
"C19_NE24F4P", # Fam 4 enfants ou plus moins 25 ans en 2019 (compl)
"P19_POP0205", # Pop 2-5 ans en 2019 (princ)
"P19_POP0610", # Pop 6-10 ans en 2019 (princ)
"P19_POP1114", # Pop 11-14 ans en 2019 (princ)
"P19_POP1517", # Pop 15-17 ans en 2019 (princ)
"P19_POP1824", # Pop 18-24 ans en 2019 (princ)
"P19_POP2529", # Pop 25-29 ans en 2019 (princ)
"P19_POP30P", # Pop 30 ans ou plus en 2019 (princ)
"P19_SCOL0205", # Pop scolarisée 2-5 ans en 2019 (princ)
"P19_SCOL0610", # Pop scolarisée 6-10 ans en 2019 (princ)
"P19_SCOL1114", # Pop scolarisée 11-14 ans en 2019 (princ)
"P19_SCOL1517", # Pop scolarisée 15-17 ans en 2019 (princ)
"P19_SCOL1824", # Pop scolarisée 18-24 ans en 2019 (princ)
"P19_SCOL2529", # Pop scolarisée 25-29 ans en 2019 (princ)
"P19_SCOL30P", # Pop scolarisée 30 ans ou plus en 2019 (princ)
"P19_NSCOL15P", # Pop 15 ans ou plus non scolarisée en 2019 (princ)
"P19_NSCOL15P_DIPLMIN", # Pop 15 ans ou plus non scol. Sans diplôme ou CEP en 2019 (princ)
"P19_NSCOL15P_BEPC", # Pop 15 ans ou plus non scol. BEPC, brevet des collèges, DNB en 2019 (princ)
"P19_NSCOL15P_CAPBEP", # Pop 15 ans ou plus non scol. CAP-BEP ou équiv. en 2019 (princ)
"P19_NSCOL15P_BAC", # Pop 15 ans ou plus non scol. Bac, brevet pro. ou équiv. en 2019 (princ)
"P19_NSCOL15P_SUP2", # Pop 15 ans ou plus non scol. Enseignement sup de niveau bac + 2 en 2019 (princ)
"P19_NSCOL15P_SUP34", # Pop 15 ans ou plus non scol. Enseignement sup de niveau bac + 3 ou 4 en 2019 (princ)
"P19_NSCOL15P_SUP5", # Pop 15 ans ou plus non scol. Enseignement sup de niveau bac + 5 ou plus en 2019 (princ)
"P19_HNSCOL15P", # Hommes 15 ans ou plus non scolarisée en 2019 (princ)
"P19_HNSCOL15P_DIPLMIN", # Hommes 15 ans ou plus non scol. Sans diplôme ou CEP en 2019 (princ)
"P19_HNSCOL15P_BEPC", # Hommes 15 ans ou plus non scol. BEPC, brevet des collèges, DNB en 2019 (princ)
"P19_HNSCOL15P_CAPBEP", # Hommes 15 ans ou plus non scol. CAP-BEP ou équiv. en 2019 (princ)
"P19_HNSCOL15P_BAC", # Hommes 15 ans ou plus non scol. Bac, brevet pro. ou équiv. en 2019 (princ)
"P19_HNSCOL15P_SUP2", # Hommes 15 ans ou plus non scol. Enseignement sup de niveau bac + 2 en 2019 (princ)
"P19_HNSCOL15P_SUP34", # Hommes 15 ans ou plus non scol. Enseignement sup de niveau bac + 3 ou 4 en 2019 (princ)
"P19_HNSCOL15P_SUP5", # Hommes 15 ans ou plus non scol. Enseignement sup de niveau bac + 5 ou plus en 2019 (princ)
"P19_FNSCOL15P", # Femmes 15 ans ou plus non scolarisée en 2019 (princ)
"P19_FNSCOL15P_DIPLMIN", # Femmes 15 ans ou plus non scol. Sans diplôme ou CEP en 2019 (princ)
"P19_FNSCOL15P_BEPC", # Femmes 15 ans ou plus non scol. BEPC, brevet des collèges, DNB en 2019 (princ)
"P19_FNSCOL15P_CAPBEP", # Femmes 15 ans ou plus non scol. CAP-BEP ou équiv. en 2019 (princ)
"P19_FNSCOL15P_BAC", # Femmes 15 ans ou plus non scol. Bac, brevet pro. ou équiv. en 2019 (princ)
"P19_FNSCOL15P_SUP2", # Femmes 15 ans ou plus non scol. Enseignement sup de niveau bac + 2 en 2019 (princ)
"P19_FNSCOL15P_SUP34", # Femmes 15 ans ou plus non scol. Enseignement sup de niveau bac + 3 ou 4 en 2019 (princ)
"P19_FNSCOL15P_SUP5", # Femmes 15 ans ou plus non scol. Enseignement sup de niveau bac + 5 ou plus en 2019 (princ)
"P19_POP", # Population en 2019 (princ)
"P19_POP0002", # Pop 0-2 ans en 2019 (princ)
"P19_POP0305", # Pop 3-5 ans en 2019 (princ)
"P19_POP0610", # Pop 6-10 ans en 2019 (princ)
"P19_POP1117", # Pop 11-17 ans en 2019 (princ)
"P19_POP1824", # Pop 18-24 ans en 2019 (princ)
"P19_POP2539", # Pop 25-39 ans en 2019 (princ)
"P19_POP4054", # Pop 40-54 ans en 2019 (princ)
"P19_POP5564", # Pop 55-64 ans en 2019 (princ)
"P19_POP6579", # Pop 65-79 ans en 2019 (princ)
"P19_POP80P", # Pop 80 ans ou plus en 2019 (princ)
"P19_POP0014", # Pop 0-14 ans en 2019 (princ)
"P19_POP1529", # Pop 15-29 ans en 2019 (princ)
"P19_POP3044", # Pop 30-44 ans en 2019 (princ)
"P19_POP4559", # Pop 45-59 ans en 2019 (princ)
"P19_POP6074", # Pop 60-74 ans en 2019 (princ)
"P19_POP75P", # Pop 75 ans ou plus en 2019 (princ)
"P19_POP0019", # Pop 0-19 ans en 2019 (princ)
"P19_POP2064", # Pop 20-64 ans en 2019 (princ)
"P19_POP65P", # Pop 65 ans ou plus en 2019 (princ)
"P19_POPH", # Pop Hommes en 2019 (princ)
"P19_H0014", # Pop Hommes 0-14 ans en 2019 (princ)
"P19_H1529", # Pop Hommes 15-29 ans en 2019 (princ)
"P19_H3044", # Pop Hommes 30-44 ans en 2019 (princ)
"P19_H4559", # Pop Hommes 45-59 ans en 2019 (princ)
"P19_H6074", # Pop Hommes 60-74 ans en 2019 (princ)
"P19_H75P", # Pop Hommes 75 ans ou plus en 2019 (princ)
"P19_H0019", # Pop Hommes 0-19 ans en 2019 (princ)
"P19_H2064", # Pop Hommes 20-64 ans en 2019 (princ)
"P19_H65P", # Pop Hommes 65 ans ou plus en 2019 (princ)
"P19_POPF", # Pop Femmes en 2019 (princ)
"P19_F0014", # Pop Femmes 0-14 ans en 2019 (princ)
"P19_F1529", # Pop Femmes 15-29 ans en 2019 (princ)
"P19_F3044", # Pop Femmes 30-44 ans en 2019 (princ)
"P19_F4559", # Pop Femmes 45-59 ans en 2019 (princ)
"P19_F6074", # Pop Femmes 60-74 ans en 2019 (princ)
"P19_F75P", # Pop Femmes 75 ans ou plus en 2019 (princ)
"P19_F0019", # Pop Femmes 0-19 ans en 2019 (princ)
"P19_F2064", # Pop Femmes 20-64 ans en 2019 (princ)
"P19_F65P", # Pop Femmes 65 ans ou plus en 2019 (princ)
"C19_POP15P", # Pop 15 ans ou plus en 2019 (compl)
"C19_POP15P_CS1", # Pop 15 ans ou plus Agriculteurs exploitants en 2019 (compl)
"C19_POP15P_CS2", # Pop 15 ans ou plus Artisans, Comm., Chefs entr. en 2019 (compl)
"C19_POP15P_CS3", # Pop 15 ans ou plus Cadres, Prof. intel. sup. en 2019 (compl)
"C19_POP15P_CS4", # Pop 15 ans ou plus Prof. intermédiaires en 2019 (compl)
"C19_POP15P_CS5", # Pop 15 ans ou plus Employés en 2019 (compl)
"C19_POP15P_CS6", # Pop 15 ans ou plus Ouvriers en 2019 (compl)
"C19_POP15P_CS7", # Pop 15 ans ou plus Retraités en 2019 (compl)
"C19_POP15P_CS8", # Pop 15 ans ou plus Autres en 2019 (compl)
"C19_H15P", # Pop 15 ans ou plus Hommes en 2019 (compl)
"C19_H15P_CS1", # Pop 15 ans ou plus Hommes Agriculteurs exploitants en 2019 (compl)
"C19_H15P_CS2", # Pop 15 ans ou plus Hommes Artisans, Comm., Chefs entr. en 2019 (compl)
"C19_H15P_CS3", # Pop 15 ans ou plus Hommes Cadres, Prof. intel. sup. en 2019 (compl)
"C19_H15P_CS4", # Pop 15 ans ou plus Hommes Prof. intermédiaires en 2019 (compl)
"C19_H15P_CS5", # Pop 15 ans ou plus Hommes Employés en 2019 (compl)
"C19_H15P_CS6", # Pop 15 ans ou plus Hommes Ouvriers en 2019 (compl)
"C19_H15P_CS7", # Pop 15 ans ou plus Hommes Retraités en 2019 (compl)
"C19_H15P_CS8", # Pop 15 ans ou plus Hommes Autres en 2019 (compl)
"C19_F15P", # Pop 15 ans ou plus Femmes en 2019 (compl)
"C19_F15P_CS1", # Pop 15 ans ou plus Femmes Agriculteurs exploitants en 2019 (compl)
"C19_F15P_CS2", # Pop 15 ans ou plus Femmes Artisans, Comm., Chefs entr. en 2019 (compl)
"C19_F15P_CS3", # Pop 15 ans ou plus Femmes Cadres, Prof. intel. sup. en 2019 (compl)
"C19_F15P_CS4", # Pop 15 ans ou plus Femmes Prof. intermédiaires en 2019 (compl)
"C19_F15P_CS5", # Pop 15 ans ou plus Femmes Employés en 2019 (compl)
"C19_F15P_CS6", # Pop 15 ans ou plus Femmes Ouvriers en 2019 (compl)
"C19_F15P_CS7", # Pop 15 ans ou plus Femmes Retraités en 2019 (compl)
"C19_F15P_CS8", # Pop 15 ans ou plus Femmes Autres en 2019 (compl)
"P19_POP_FR", # Pop Français en 2019 (princ)
"P19_POP_ETR", # Pop Etrangers en 2019 (princ)
"P19_POP_IMM", # Pop Immigrés en 2019 (princ)
"P19_PMEN", # Pop ménages en 2019 (princ)
"P19_PHORMEN", # Pop hors ménages en 2019 (princ)
"P19_LOG", # Logements en 2019 (princ)
"P19_RP", # Résidences principales en 2019 (princ)
"P19_RSECOCC", # Rés secondaires et logts occasionnels en 2019 (princ)
"P19_LOGVAC", # Logements vacants en 2019 (princ)
"P19_MAISON", # Maisons en 2019 (princ)
"P19_APPART", # Appartements en 2019 (princ)
"P19_RP_1P", # Rés princ 1 pièce en 2019 (princ)
"P19_RP_2P", # Rés princ 2 pièces en 2019 (princ)
"P19_RP_3P", # Rés princ 3 pièces en 2019 (princ)
"P19_RP_4P", # Rés princ 4 pièces en 2019 (princ)
"P19_RP_5PP", # Rés princ 5 pièces ou plus en 2019 (princ)
"P19_NBPI_RP", # Pièces rés princ en 2019 (princ)
"P19_RPMAISON", # Rés princ type maison en 2019 (princ)
"P19_NBPI_RPMAISON", # Pièces rés princ type maison en 2019 (princ)
"P19_RPAPPART", # Rés princ type appartement en 2019 (princ)
"P19_NBPI_RPAPPART", # Pièces rés princ type appartement en 2019 (princ)
"P19_RP_M30M2", # Rés princ de moins de 30 m2 en 2019 (princ)
"P19_RP_3040M2", # Rés princ de 30 à moins de 40 m2 en 2019 (princ)
"P19_RP_4060M2", # Rés princ de 40 à moins de 60 m2 en 2019 (princ)
"P19_RP_6080M2", # Rés princ de 60 à moins de 80 m2 en 2019 (princ)
"P19_RP_80100M2", # Rés princ de 80 à moins de 100 m2 en 2019 (princ)
"P19_RP_100120M2", # Rés princ de 100 à moins de 120 m2 en 2019 (princ)
"P19_RP_120M2P", # Rés princ de 120 m2 ou plus en 2019 (princ)
"P19_RP_ACHTOT", # Rés princ avt 2016 en 2019 (princ) 
"P19_RP_ACH19", # Rés princ avt 1919 en 2019 (princ) 
"P19_RP_ACH45", # Rés princ 1919 à 1945 en 2019 (princ) 
"P19_RP_ACH70", # Rés princ 1946 à 1970 en 2019 (princ) 
"P19_RP_ACH90", # Rés princ 1971 à 1990 en 2019 (princ) 
"P19_RP_ACH05", # Rés princ 1991 à 2005 en 2019 (princ) 
"P19_RP_ACH15", # Rés princ 2006 à 2015 en 2019 (princ) 
"P19_RPMAISON_ACHTOT", # Rés princ Type maison avt 2016 en 2019 (princ) 
"P19_RPMAISON_ACH19", # Rés princ Type maison avt 1919 en 2019 (princ) 
"P19_RPMAISON_ACH45", # Rés princ Type maison 1919 à 1945 en 2019 (princ) 
"P19_RPMAISON_ACH70", # Rés princ Type maison 1946 à 1970 en 2019 (princ) 
"P19_RPMAISON_ACH90", # Rés princ Type maison 1971 à 1990 en 2019 (princ) 
"P19_RPMAISON_ACH05", # Rés princ Type maison 1991 à 2005 en 2019 (princ) 
"P19_RPMAISON_ACH15", # Rés princ Type maison 2006 à 2015 en 2019 (princ) 
"P19_RPAPPART_ACHTOT", # Rés princ Type appart avt 2016 en 2019 (princ) 
"P19_RPAPPART_ACH19", # Rés princ Type appart avt 1919 en 2019 (princ) 
"P19_RPAPPART_ACH45", # Rés princ Type appart 1919 à 1945 en 2019 (princ) 
"P19_RPAPPART_ACH70", # Rés princ Type appart 1946 à 1970 en 2019 (princ) 
"P19_RPAPPART_ACH90", # Rés princ Type appart 1971 à 1990 en 2019 (princ) 
"P19_RPAPPART_ACH05", # Rés princ Type appart 1991 à 2005 en 2019 (princ) 
"P19_RPAPPART_ACH15", # Rés princ Type appart 2006 à 2015 en 2019 (princ) 
"P19_MEN", # Ménages en 2019 (princ)
"P19_MEN_ANEM0002", # Ménages emménagés moins 2 ans en 2019 (princ)
"P19_MEN_ANEM0204", # Ménages emménagés entre 2-4 ans en 2019 (princ)
"P19_MEN_ANEM0509", # Ménages emménagés entre 5-9 ans en 2019 (princ)
"P19_MEN_ANEM10P", # Ménages emménagés depuis 10 ans ou plus en 2019 (princ)
"P19_PMEN", # Pop ménages en 2019 (princ)
"P19_PMEN_ANEM0002", # Pop mén emménagés moins 2 ans en 2019 (princ)
"P19_PMEN_ANEM0204", # Pop mén emménagés entre 2-4 ans en 2019 (princ)
"P19_PMEN_ANEM0509", # Pop mén emménagés entre 5-9 ans en 2019 (princ)
"P19_PMEN_ANEM10P", # Pop mén emménagés depuis 10 ans ou plus en 2019 (princ)
"P19_NBPI_RP_ANEM0002", # Pièces Rés princ Mén. emménagés moins 2 ans en 2019 (princ)
"P19_NBPI_RP_ANEM0204", # Pièces Rés princ Mén. emménagés entre 2-4 ans en 2019 (princ)
"P19_NBPI_RP_ANEM0509", # Pièces Rés princ Mén. emménagés entre 5-9 ans en 2019 (princ)
"P19_NBPI_RP_ANEM10P", # Pièces Rés princ Mén. emménagés depuis 10 ans ou plus en 2019 (princ)
"P19_RP_PROP", # Rés princ occupées Propriétaires en 2019 (princ)
"P19_RP_LOC", # Rés princ occupées Locataires en 2019 (princ)
"P19_RP_LOCHLMV", # Rés princ HLM louée vide en 2019 (princ)
"P19_RP_GRAT", # Rés princ logé gratuit en 2019 (princ)
"P19_NPER_RP", # Personnes Rés princ en 2019 (princ)
"P19_NPER_RP_PROP", # Pers Rés princ occupées Propriétaires en 2019 (princ)
"P19_NPER_RP_LOC", # Pers Rés princ occupées Locataires en 2019 (princ)
"P19_NPER_RP_LOCHLMV", # Pers Rés princ HLM louées vides en 2019 (princ)
"P19_NPER_RP_GRAT", # Pers Rés princ occupées gratuit en 2019 (princ)
"P19_ANEM_RP", # Anc tot Emméngt Rés princ (années) en 2019 (princ)
"P19_ANEM_RP_PROP", # Anc tot Emméngt Rés princ occ par Propriétaires (années) en 2019 (princ)
"P19_ANEM_RP_LOC", # Anc tot Emméngt Rés princ occ par Locataires (années) en 2019 (princ)
"P19_ANEM_RP_LOCHLMV", # Anc tot Emméngt Rés princ HLM louées vides (années) en 2019 (princ)
"P19_ANEM_RP_GRAT", # Anc tot Emméngt Rés princ occ gratuit (années) en 2019 (princ)
"P19_RP_SDB", # Rés princ SDB baignoire douche (MET) en 2019 (princ)
"P19_RP_CCCOLL", # Rés princ Chauffage Central Collectif (MET) en 2019 (princ)
"P19_RP_CCIND", # Rés princ Chauffage Central Individuel (MET) en 2019 (princ)
"P19_RP_CINDELEC", # Rés princ Chauffage Individuel Electrique (MET) en 2019 (princ)
"P19_RP_GARL", # Ménages au moins un parking en 2019 (princ)
"P19_RP_VOIT1P", # Ménages au moins une voiture en 2019 (princ)
"P19_RP_VOIT1", # Ménages une voiture en 2019 (princ)
"P19_RP_VOIT2P", # Ménages deux voitures ou plus en 2019 (princ)
"C19_RP_HSTU1P", # Rés princ hors studio de 1 personne en 2019 (compl)
"C19_RP_HSTU1P_SUROCC", # Rés princ hors studio de 1 personne en suroccupation en 2019 (compl)
]

In [42]:
geoiris['surface'] = geoiris['geometry'].area
for index in range(len(variables)):
    variable = variables[index]
    print(f"{index+1}/{len(variables)} : {variable}")
    if variable not in geoiris:
        print(f"   => La variable n'est pas dans geoiris")
    else:
        geoiris['densité'] = geoiris[variable] / geoiris['surface']
        df_geom[variable] = 0.0
        for index, geom_row in df_geom.iterrows():
            for _, iris_row in geoiris.iterrows():
                # Calculer l'intersection
                intersection = geom_row.geometry.intersection(iris_row.geometry)
                # Vérifier si l'intersection est non vide
                if not intersection.is_empty:
                    # Calculer la surface de l'intersection
                    surface_intersection = intersection.area
                    # Ajouter les logements proportionnels à df_geom
                    df_geom.at[index, variable] += surface_intersection * iris_row['densité']

1/371 : P19_POP1564
2/371 : P19_POP1524
3/371 : P19_POP2554
4/371 : P19_POP5564
5/371 : P19_H1564
6/371 : P19_H1524
7/371 : P19_H2554
8/371 : P19_H5564
9/371 : P19_F1564
10/371 : P19_F1524
11/371 : P19_F2554
12/371 : P19_F5564
13/371 : P19_ACT1564
14/371 : P19_ACT1524
15/371 : P19_ACT2554
16/371 : P19_ACT5564
17/371 : P19_HACT1564
18/371 : P19_HACT1524
19/371 : P19_HACT2554
20/371 : P19_HACT5564
21/371 : P19_FACT1564
22/371 : P19_FACT1524
23/371 : P19_FACT2554
24/371 : P19_FACT5564
25/371 : P19_ACTOCC1564
26/371 : P19_ACTOCC1524
27/371 : P19_ACTOCC2554
28/371 : P19_ACTOCC5564
29/371 : P19_HACTOCC1564
30/371 : P19_HACTOCC1524
31/371 : P19_HACTOCC2554
32/371 : P19_HACTOCC5564
33/371 : P19_FACTOCC1564
34/371 : P19_FACTOCC1524
35/371 : P19_FACTOCC2554
36/371 : P19_CHOM_CAPBEP
37/371 : P19_FACTOCC5564
38/371 : P19_CHOM_BAC
39/371 : P19_CHOM1564
40/371 : P19_CHOM_SUP2
41/371 : P19_CHOM1524
42/371 : P19_CHOM_SUP34
43/371 : P19_CHOM2554
44/371 : P19_CHOM_SUP5
45/371 : P19_CHOM5564
46/371 : P19

In [43]:
df_geom.drop('geometry', axis=1, inplace=True)

In [44]:
df_geom.head(3)

,type,nom,code_geom,surface,longitude,latitude,surface_Lande,surface_Forêt fermée sans couvert arboré,surface_Forêt fermée de feuillus purs en îlots,surface_Forêt fermée à mélange de feuillus,...,P19_ANEM_RP_PROP,P19_ANEM_RP_LOC,P19_ANEM_RP_LOCHLMV,P19_ANEM_RP_GRAT,P19_RP_GARL,P19_RP_VOIT1P,P19_RP_VOIT1,P19_RP_VOIT2P,C19_RP_HSTU1P,C19_RP_HSTU1P_SUROCC
0,DEP,Yvelines,0,2.306381e+09,614927.055009,6.857846e+06,1.332016e+07,1.205007e+07,1.945727e+07,3.292565e+08,...,6.315476e+06,2.202992e+06,1.477552e+06,138508.803303,412439.852395,496303.309180,292006.514108,204296.795072,555140.258122,39378.858094
1,CIS,HOI,1,4.299060e+07,642209.966787,6.872238e+06,0.000000e+00,0.000000e+00,0.000000e+00,8.815444e+03,...,3.799461e+05,1.423648e+05,9.988902e+04,7858.103922,22856.698401,27041.673751,18450.787939,8590.885811,33124.109082,3301.444617
2,CIS,VLY,2,2.313954e+07,639271.615586,6.852965e+06,0.000000e+00,2.103211e+05,4.222985e+04,2.660572e+06,...,1.504026e+05,7.686775e+04,3.243861e+04,3940.649816,9128.571102,12887.439114,7880.340632,5007.098482,13724.135601,702.191636


In [45]:
from sklearn.preprocessing import StandardScaler

col_to_normalize = [col for col in df_geom if col not in ['type', 'nom', 'code_geom']]
scaler = StandardScaler()
df_geom[col_to_normalize] = scaler.fit_transform(df_geom[col_to_normalize])

In [46]:
df_geom.to_feather

<bound method GeoDataFrame.to_feather of    type       nom  code_geom   surface  longitude  latitude  surface_Lande  \
0   DEP  Yvelines          0  6.327224  -0.627456 -0.342520       6.135828   
1   CIS       HOI          1 -0.216998   1.452430  0.721882      -0.308230   
2   CIS       VLY          2 -0.274394   1.228428 -0.703505      -0.308230   
3   CIS       VIL          3 -0.220679   0.338604 -0.124399      -0.305254   
4   CIS       AUB          4 -0.172808  -0.606183  0.794807      -0.079330   
5   CIS       CLV          5 -0.247782   0.409573  0.951609       0.231330   
6   CIS       BOI          6 -0.288406   0.500520 -0.417823      -0.253572   
7   CIS       MLB          7 -0.263296   0.275570 -0.698975      -0.308230   
8   CIS       VES          8 -0.317927   0.990488  0.254906      -0.308230   
9   CIS       VIR          9 -0.326827   1.235709 -0.466717      -0.308230   
10  CIS       BON         10  0.166641  -2.171931  1.617758       1.315629   
11  CIS       MLF      